## Setup

*You must run the cells in this section each time you connect to a new runtime. For example, when you return to the notebook after an idle timeout, when the runtime crashes, or when you restart or factory reset the runtime.*

Set name of spreadsheet to export results to:

In [ ]:
spreadsheet_name = 'feedback_results'

Enter credentials:

In [ ]:
# Get database credentials. OCDS Helpdesk analysts, see https://crm.open-contracting.org/issues/6335.
import getpass

print('Enter your Kingfisher credentials')
user = input('Username:')
password = getpass.getpass('Password:')

Setup notebook environment:

In [ ]:
# Install Kingfisher Colab and required packages
%shell pip install --upgrade 'ocdskingfishercolab<0.4' psycopg2-binary > pip.log

# Import libraries and functions
from ocdskingfishercolab import (
    list_source_ids,
    list_collections,
    set_spreadsheet_name,
    save_dataframe_to_sheet,
    save_dataframe_to_spreadsheet,
    set_search_path)
import pandas as pd

# Load https://pypi.org/project/ipython-sql/
%load_ext sql 

# Load https://colab.research.google.com/notebooks/data_table.ipynb
%load_ext google.colab.data_table 

# Set config
%config SqlMagic.autopandas = True  # Return Pandas DataFrames instead of regular result sets
%config SqlMagic.displaycon = False  # Don't show connection string after execute
%config SqlMagic.feedback = False  # Don't print number of rows affected by DML
set_spreadsheet_name(spreadsheet_name) # Set name of spreadsheet for exporting results

# Connect to database
connection_string = 'postgresql://' + user + ':' + password + '@postgres-readonly.kingfisher.open-contracting.org/ocdskingfisherprocess?sslmode=require'
%sql $connection_string

# Define function for rendering collapsible JSON in a notebook
# Remove once https://github.com/open-contracting/kingfisher-colab/issues/40 is done
import json
from IPython.display import HTML

def render_json(jstr):
  if type(jstr) != str:
    jstr = json.dumps(jstr)
  return HTML("""
<script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
<script>
renderjson.set_show_to_level(1)
document.body.appendChild(renderjson(%s))
new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
</script>
""" % jstr)
  
# Install and setup plotting library
# Maybe this can also be moved to Kingfisher-Colab?
!pip install seaborn

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr

colab_dark_style = {'figure.facecolor': '#383838',
                       'axes.edgecolor': '#d5d5d5',
                       'axes.facecolor': '#383838',
                       'axes.labelcolor': '#d5d5d5',
                       'text.color': '#d5d5d5',
                       'xtick.color': '#d5d5d5',
                       'ytick.color': '#d5d5d5'}

sns.set_style('dark', colab_dark_style)

# Define function to apply number formatting to axis labels
# Maybe this can also be moved to Kingfisher-Colab?
# Needs updating to support other locales
def format_thousands(axis):
  axis.set_major_formatter(tkr.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))